In [9]:
%matplotlib inline
%load_ext ipycache

import pandas as pd
import numpy as np
import scipy
import sklearn as sk
import xgboost as xgb

import seaborn as sns
sns.set()

from vowpalwabbit.sklearn_vw import VWRegressor

import matplotlib.pyplot as plt

The ipycache extension is already loaded. To reload it, use:
  %reload_ext ipycache


ImportError: No module named seaborn

In [2]:
import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [
        (math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 
        for i,pred in enumerate(y_pred)
    ]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

# Загрузка данных

In [55]:
# train_raw = pd.read_csv("data/train.csv")
train_raw = pd.read_csv("data/train_without_noise.csv")
test = pd.read_csv("data/test.csv")
macro = pd.read_csv("data/macro.csv")
train_raw.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


# Препроцессинг фич

In [56]:
def preprocess(df):
#     df = df.merge(macro, on="timestamp")
    from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
    df = df.drop(["id", "timestamp", "sub_area"], axis=1)

    onehot_feat = ["product_type"]
    # onehot_feat = map(list(train.columns).index, onehot_feat)
    pre_steps = [
    #     OneHotEncoder(categorical_features=onehot_feat)
    ]
    df = df.drop(onehot_feat, axis=1)

    ecology = ["no data", "poor", "satisfactory", "good", "excellent"]
    df["ecology"] = map(ecology.index, df["ecology"].values)
    
    bool_feats = [
        "thermal_power_plant_raion",
        "incineration_raion",
        "oil_chemistry_raion",
        "radiation_raion",
        "railroad_terminal_raion",
        "big_market_raion",
        "nuclear_reactor_raion",
        "detention_facility_raion",
        "water_1line",
        "big_road1_1line",
        "railroad_1line",
        "culture_objects_top_25"
    ]
    for bf in bool_feats:
        df[bf] = map(lambda x: x == "yes", df[bf].values)
    
    df = df.select_dtypes(exclude=['object'])
    df["full_sq"] = map(lambda x: x if x > 10 else float("NaN"), df["full_sq"])
    df["life_sq"] = map(lambda x: x if x > 5 else float("NaN"), df["life_sq"])
    df["kitch_sq"] = map(lambda x: x if x > 2 else float("NaN"), df["kitch_sq"])

#     full_sq-life_sq<0 full_sq-kitch_sq<0 life_sq-kitch_sq<0 floor-max_floor<0

#     df = df.replace(float("NaN"), -1)

#     df["groud_floor"] = map(lambda x: 0 < x <= 2, df["floor"].values)
    return df

In [57]:
train = preprocess(train_raw)
X = train.drop(["price_doc"], axis=1)
y = train[["price_doc"]]

# Обучение моделей

In [58]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold

def validate(clf):
    cval = cross_val_score(clf, X.values, y.values, cv=3, scoring=make_scorer(rmsle, False), verbose=2)
    return np.mean(cval), cval

In [61]:
model = xgb.XGBRegressor(max_depth=10, n_estimators=800, learning_rate=0.01, nthread=-1)
print validate(model)

[CV]  ................................................................
[CV] ................................................. , total= 3.1min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.1min remaining:    0.0s


[CV] ................................................. , total= 2.9min
[CV]  ................................................................
[CV] ................................................. , total= 3.0min
(-0.49697011042311873, array([-0.5541983 , -0.48005079, -0.45666124]))


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  9.1min finished


In [38]:
model = VWRegressor()
model.fit(train_X, train_y)

print rmsle(test_y, model.predict(test_X))

15.6186260067


# Submission

In [62]:
test_pr = preprocess(test)

model.fit(X.values, y.values)

submdf = pd.DataFrame({"id": test["id"], "price_doc": model.predict(test_pr.values)})
submdf.to_csv("/tmp/sber_submission.csv", header=True, index=False)
!head /tmp/sber_submission.csv

id,price_doc
30474,5690930.0
30475,8467048.0
30476,5846252.5
30477,6100667.5
30478,5201817.0
30479,9344886.0
30480,4365896.0
30481,4144179.75
30482,4997950.0
